# Loading Marathi Dataset

The dataset for marathi language is directly fetched from the website.

In [1]:
!pip install datasets
from datasets import ClassLabel, Sequence, Dataset, Value, Features, load_dataset, load_metric
raw_datasets = load_dataset('ai4bharat/naamapadam','mr')

  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
features = raw_datasets["train"].features
label_list = features['ner_tags'].feature.names
label_to_id = {label_list[i]: features['ner_tags'].feature.str2int( label_list[i] ) for i in range(len(label_list))}
print(label_to_id)
num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


# FineTuning IndicBERT for NER classification

Firstly, we need to tokenize the training and validation dataset and then map the words to their NER labels.

In [3]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

2024-03-13 12:24:59.136213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 12:24:59.136272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 12:24:59.137883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [4]:
model=model.to("cuda")

In [5]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples['ner_tags']):      
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:      
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


## Tokenizing train_dataset

In [6]:
train_dataset = raw_datasets["train"].select(range(20000))   #sampling

train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

## Tokenizing validation_dataset

In [7]:
eval_dataset = raw_datasets["validation"]        #no-sampling
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
data_collator = DataCollatorForTokenClassification(tokenizer)

## Designing a compute metrics

We require compute metrics to evaluate the model. I have retrieved precision, recall, f1-score and accuracy here. Using these values, I will calculate the macro-f1 score.

In [9]:
!pip install seqeval

def all_classes(true_tags, pred_tags):
    classes = set(tag for row in true_tags + pred_tags for tag in row)

    results = []
    true_positives, false_positives, false_negatives = 0, 0, 0
    for class_label in sorted(classes):
        true_indices = [(i, j) for i, row in enumerate(true_tags) for j, tag in enumerate(row) if tag == class_label]
        pred_indices = [(i, j) for i, row in enumerate(pred_tags) for j, tag in enumerate(row) if tag == class_label]
        
        true_positive = len(set(true_indices) & set(pred_indices))
        false_positive = len(set(pred_indices) - set(true_indices))
        false_negative = len(set(true_indices) - set(pred_indices))

        precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
        recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        results.append((class_label, precision, recall, f1_score))

        true_positives += true_positive
        false_positives += false_positive
        false_negatives += false_negative

    macro_f1_score = 2 * (true_positives) / (2 * true_positives + false_positives + false_negatives) if (2 * true_positives + false_positives + false_negatives) > 0 else 0
    results.append(('macro f1:', 0, 0, macro_f1_score))

    return results




In [10]:
# Metrics function
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    r1 = all_classes(true_labels, true_predictions)
    print("Class\tPrecision\tRecall\tF1-Score")
    for class_label, precision, recall, f1_score in r1:
        print(f"{class_label}\t{precision:.2f}\t\t{recall:.2f}\t{f1_score:.2f}")
        
    results = metric.compute(predictions=true_predictions, references=true_labels)

    return results

In [11]:
!pip install accelerate -U

  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


## Finetuning the hyper-parameters

For fine-tuning a pre-trained model, we need to tune the hyper-parameters which are mentioned in the TrainingArguments. If a particular hyper parameter is not mentioned then it will take the default value.

In [12]:
args=TrainingArguments(
    output_dir='Trained Models/IndicBERT/Fine_tuned_IndicBERT',
    num_train_epochs=3,
    logging_steps=500, #500
    save_steps=1000, #1000
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,)

## Training the model

As training was taking more time and due to limited resources, I sampled off the training dataset. Also, I discussed this issue with my colleagues. They were getting different EFT (Expected Finish Time).

In [13]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,    #.select(range(20000)), #sampling
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
)

In [14]:
#GPU 100,000 -> 12h , 10k -> 3h, entire dataset -> 61h, 25% -> 15h
#GPU Kaggle p100: 100,000 -> 7h,
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.665600
1000,0.458200
1500,0.405500
2000,0.360800
2500,0.357200
3000,0.294100
3500,0.280900


Checkpoint destination directory Trained Models/IndicBERT/Fine_tuned_IndicBERT/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory Trained Models/IndicBERT/Fine_tuned_IndicBERT/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory Trained Models/IndicBERT/Fine_tuned_IndicBERT/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=3750, training_loss=0.39502447509765626, metrics={'train_runtime': 3120.8936, 'train_samples_per_second': 19.225, 'train_steps_per_second': 1.202, 'total_flos': 1325736898560000.0, 'train_loss': 0.39502447509765626, 'epoch': 3.0})

In [15]:
trainer.evaluate(train_dataset)
#takes time

Class	Precision	Recall	F1-Score
B-LOC	0.81		0.86	0.84
B-ORG	0.82		0.73	0.77
B-PER	0.87		0.85	0.86
I-LOC	0.73		0.52	0.61
I-ORG	0.80		0.65	0.71
I-PER	0.89		0.87	0.88
O	0.95		0.97	0.96
macro f1:	0.00		0.00	0.92


Trainer is attempting to log a value of "{'precision': 0.7891179918626302, 'recall': 0.8429776419020958, 'f1': 0.8151591245035706, 'number': 27149}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7646996391424326, 'recall': 0.696606400464082, 'f1': 0.7290665317480394, 'number': 20686}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8435242681535927, 'recall': 0.8291809405169729, 'f1': 0.8362911078305117, 'number': 32110}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.25029969215393066,
 'eval_LOC': {'precision': 0.7891179918626302,
  'recall': 0.8429776419020958,
  'f1': 0.8151591245035706,
  'number': 27149},
 'eval_ORG': {'precision': 0.7646996391424326,
  'recall': 0.696606400464082,
  'f1': 0.7290665317480394,
  'number': 20686},
 'eval_PER': {'precision': 0.8435242681535927,
  'recall': 0.8291809405169729,
  'f1': 0.8362911078305117,
  'number': 32110},
 'eval_overall_precision': 0.8049489988666415,
 'eval_overall_recall': 0.7995621990118206,
 'eval_overall_f1': 0.8022465564306109,
 'eval_overall_accuracy': 0.9247723389285355,
 'eval_runtime': 419.7984,
 'eval_samples_per_second': 47.642,
 'eval_steps_per_second': 2.978,
 'epoch': 3.0}

In [16]:
#GPU
trainer.evaluate() #evaluation values


Class	Precision	Recall	F1-Score
B-LOC	0.75		0.78	0.76
B-ORG	0.72		0.68	0.70
B-PER	0.81		0.79	0.80
I-LOC	0.59		0.46	0.52
I-ORG	0.63		0.52	0.57
I-PER	0.84		0.83	0.83
O	0.93		0.95	0.94
macro f1:	0.00		0.00	0.89


Trainer is attempting to log a value of "{'precision': 0.7256116444719728, 'recall': 0.7533762057877813, 'f1': 0.7392333175579744, 'number': 3110}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.6663996796155387, 'recall': 0.6457120682964688, 'f1': 0.6558927867560111, 'number': 2577}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7661768497035318, 'recall': 0.7533586818757921, 'f1': 0.7597137014314929, 'number': 3945}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.36112767457962036,
 'eval_LOC': {'precision': 0.7256116444719728,
  'recall': 0.7533762057877813,
  'f1': 0.7392333175579744,
  'number': 3110},
 'eval_ORG': {'precision': 0.6663996796155387,
  'recall': 0.6457120682964688,
  'f1': 0.6558927867560111,
  'number': 2577},
 'eval_PER': {'precision': 0.7661768497035318,
  'recall': 0.7533586818757921,
  'f1': 0.7597137014314929,
  'number': 3945},
 'eval_overall_precision': 0.72660072878709,
 'eval_overall_recall': 0.7245639534883721,
 'eval_overall_f1': 0.7255809117845817,
 'eval_overall_accuracy': 0.8897393498235625,
 'eval_runtime': 48.27,
 'eval_samples_per_second': 47.649,
 'eval_steps_per_second': 2.983,
 'epoch': 3.0}

In [17]:
trainer.save_model("Fine_tuned_IndicBERT")

# Test Phase

In [18]:
#TEST
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/Fine_tuned_IndicBERT")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/Fine_tuned_IndicBERT")

test_dataset = raw_datasets["test"]
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/Fine_tuned_IndicBERT and are newly initialized: ['albert.pooler.bias', 'albert.pooler.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running tokenizer on test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
x = trainer.predict(test_dataset)
x.metrics

Class	Precision	Recall	F1-Score
B-LOC	0.85		0.74	0.79
B-ORG	0.78		0.68	0.73
B-PER	0.86		0.80	0.83
I-LOC	0.62		0.50	0.55
I-ORG	0.62		0.52	0.57
I-PER	0.89		0.86	0.87
O	0.93		0.97	0.95
macro f1:	0.00		0.00	0.90


{'test_loss': 0.306081622838974,
 'test_LOC': {'precision': 0.8152173913043478,
  'recall': 0.7239382239382239,
  'f1': 0.7668711656441718,
  'number': 1554},
 'test_ORG': {'precision': 0.7149576669802445,
  'recall': 0.63919259882254,
  'f1': 0.674955595026643,
  'number': 1189},
 'test_PER': {'precision': 0.8278628738147338,
  'recall': 0.7731607629427792,
  'f1': 0.7995773159563226,
  'number': 1468},
 'test_overall_precision': 0.7918196119559517,
 'test_overall_recall': 0.7171693184516742,
 'test_overall_f1': 0.7526479750778815,
 'test_overall_accuracy': 0.9039969802457745,
 'test_runtime': 22.6054,
 'test_samples_per_second': 47.776,
 'test_steps_per_second': 3.008}

In [21]:
!zip -r file.zip /kaggle/working

updating: kaggle/working/ (stored 0%)
updating: kaggle/working/.virtual_documents/ (stored 0%)
updating: kaggle/working/wandb/ (stored 0%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/ (stored 0%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/files/ (stored 0%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/files/output.log (deflated 87%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/files/config.yaml (deflated 77%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/files/wandb-metadata.json (deflated 65%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/files/wandb-summary.json (deflated 53%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/files/conda-environment.yaml (deflated 67%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/files/requirements.txt (deflated 58%)
updating: kaggle/working/wandb/run-20240312_153406-7cm37hdf/logs/ (stored 0%)
updating: kaggle/working/wandb/run-20240312

In [22]:
!ls

 Fine_tuned_IndicBERT  'Trained Models'   file.zip   state.db   wandb


In [24]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip

# Q4 Appended.

The part of Question 4 wherein we have to compute precision recall f1-score on the fine-tuned IndicBERT has been appended here.

In [25]:
Q1_data = {
    'tokens':[["कार्यकर्त्यांमधील", "वाद", "बघून", "संतापलेल्या", "सुप्रिया", "सुळे", "यांनी", "कार्यक्रमामध्येच", "राडेबाज", "कार्यकर्त्यांना","दम", "दिला", "."],
              ["अर्थात", "यात", "काही", "चांगल्या", "आणि", "व्हिडिओला", "पाठिंबा", "देणाऱ्या", "कमेंटही", "होत्या", "."],
              ["भारतीय", "वायुसेनेनं", "केलेल्या", "कारवाईचं", "संपुर्ण", "देशभरात", "कौतूक", "केलं", "जात", "आहे", "."],
              ["24)", "बैठक", "बोलावली", "असल्याची", "माहिती", "आमदार", "अनिल", "बाबर", "यांनी",".", ".", ".", "ढवळीत","पुरग्रस्तांच्या","मदत", "वाटपाची", "चौकशी", "सुरु"],
              ["यावेळी", "सुरेश", "प्रभुनी", "सुधीर", "सावंत", "व", "मधु", "दंडवते", "या", "दोघांचाही", "पराभव", "केला", ".", "त्यावेळी", "निकालादिवशी", "सुरेश", "प्रभुंना", "विजयी", "घोषित" "केल्यानंतर", "त्यांनी", "लगेचच", "शेजारी", "उभे", "असलेल्या", "मधु", "दंडवतेंची", "भेट", "घेत", "वाकुन", "त्यांच्या", "चरणांना", "स्पर्श", "केला", ".", "मतांच्या", "गोळाबेरजेत", "आपण", "विजयी", "झालेलो", "असलो", "तरी", "मधु", "दंडवतें", "या", "व्यक्तीचे", "कोकणप्रती", "असलेले", "योगदान", "प्रभुना", "चांगलेच", "ठाऊक", "होते", "."],
              ["सव्वाशे", "वर्षे", "अखंड", "पाणीपुरवठा", ":", "संस्थानकालीन", "योजना", ":", "सुधारित", "45", "कोटींचा", "आराखडा", "लवकरच", "मंजूर"],
              ["कार्यक्रम", "सर्वांना", "खुला", "असून", ",", "रसिकांनी", "उपस्थित", "रहावे", "असे", "आवाहन", "लोकमान्य", "ग्रंथालयाने", "केले", "आहे", "."],
              ["विमोचन", "आणि", "तो", "एक", "पादचारी", "दाबा", "आणि", "एक", "कार", "दाबा", "होते", "तेव्हा", ",", "उदाहरणार्थ", ",", "गुन्हा", "मागोवा", "लपविण्यासाठी", "मानले", "जाते", "."],
              ["निसर्गराजा", "मित्र", "जीवांचे", "ही", "संस्था", "गेली", "दहा", "वर्षे", "पिंपरी", "-", "चिंचवड", "परिसरातील", "तरुणांना", "एकत्र", "करून", "निसर्ग", "संवर्धनाची", "कार्ये", "विविध", "उपक्रमांद्वारे", "करत", "आहे", "."],
              ["कल्याण", "-", "डोंबिवली", "मध्ये", "माथाडी", "कामगारही", "उद्या", "बंदमध्ये", "सहभागी", "होणार", "असून", "एक", "भव्य", "मोर्चाही", "काढण्यात", "येणार", "आहे", "."],
              ["दोन्ही", "ट्रकचा", "अपघात", "एवढा", "भीषण", "होता", "की", "ट्रकच्या", "समोरील", "बाजूचा", "चक्काचुर", "होऊन", "त्यामध्ये", "ट्रकचालक", "अडकले", "होते", "."],
              ["सिबिल", "या", "संस्थेने", "प्रसिद्ध", "केलेल्या", "माहितीनुसार", ",", "2018", "मध्ये", "जाणूनबुजून", "कर्ज", "बुडविणाऱ्या", "मंडळींच्या", "संख्येत", "मोठ्या", "प्रमाणावर", "वाढ", "झाली", "आहे", "."],
              ["श्रीराम", "लागू", "-", "मराठी", "रंगभूमीचे", "अनभिषिक्त", "सम्राट", ",", "चतुरस्र", "अभिनेते", ",", "परखड", ".", ".", ".", "30", "डिसेंबरला", "होणाऱ्या", "शपथविधी", "सोहळ्याची", "विधानभवनात", "जोरदार", "तयारी", "सुरू"],
              ["कडाप्पे", "अंगावर", "पडून", "दोन", "कामगारांचा", "जागीच", "मृत्यू", ",", "चार", "जण", "गंभीर"],
              ["मात्र", "सत्ताधा-यांनी", "या", "विरोधाला", "न", "जुमानता", "हा", "प्रस्ताव", "मंजूर", "केला", "."],
              ["रजनीकांत", "यांच्यावर", "प्रेम", "करणाऱ्या", "त्याच्या", "चाहत्यांची", "संख्या", "खूप", "मोठी", "आहे", "."],
              ["एक", "हजार", "आणि", "एक", "आहेत", "नवीन", "वर्ष", "परिस्थिती", "प्रौढांसाठी", "घरी", "सर्जनशीलता", "आणि", "कल्पकता", "प्रदर्शित", ",", "आपण", "अविस्मरणीय", "काहीतरी", "अनुकूल", "कंपनी", "सामान्य", "साधना", "चालू", "शकत", "नाही", ",", "म्हणून", "."],
              ["नेपाळच्या", "सौनाली", "जवळील", "पडासरी", "भागातील", "एका", "कारखान्यात", "झालेल्या", "प्रचंड", "स्फोटांच्या", "झळा", "भारतीय", "सीमेपर्यंतही", "पोहोचल्या", "असून", "या", "भागात", "सीमा", "सुरक्षा", "दलाची", "गस्त", "वाढविली", "गेली", "आहे", "."],
              ["तसेच", "शाळांना", "लागलेल्या", "सुट्टय़ा", ",", "लग्न", "सराई", ",", "जत्रा", ",", "यात्रा", ".", ".", ".", "तीन", "ग्रामपंचायतीसाठी", "आज", "मतदान"],
              ["जग", "आपण", "सुमारे", "गोल", "फिरणे", "नाही", ",", "तो", "एक", "तारीख", "लक्षात", "विशेषतः", "महत्वाचे", "आहे", "."],
              ["देशाचे", "विभाजन", "करणाऱ्यांनाच", "नव्हे", "तर", "अशा", "लोकांशी", "हातमिळवणी", "करणाऱ्यांनाही", "देशाच्या", "सत्ताकारणात", "यापुढे", "थारा", "मिळू", "नये", "."],
              ["माजी", "जागतिक", "उपविजेता", "मुंबईचा", "दुसरा", "मानांकित", "मोहम्मद", "गुफरानने", "मुबंइ", "उपनगरच्या", "इश्तियाक", "अन्सारीचे", "25", "-", "6", ",", "25", "-", "8", "असे", "आव्हान", "परतवून", "लावले", "."],
              ["तथापि", ",", "एक", "दीर्घ", "परंपरा", "आणि", "काम", "नीतिविषयक", "लोभ", "उद्ध्वस्त", "होते", "."],
              ["सरकारने", "या", "अर्थसंकल्पीय", "अधिवेशनात", "शेतकऱ्यांना", "संपूर्ण", "कर्जमाफीची", "घोषणा", "करावी", ",", "या", "मागणीसाठी", "विरोधक", "आज", "आक्रमक", "झाले", "."],
              ["काँग्रेसचे", "विरोधक", "हे", "काँग्रेस", "मुक्त", "भारताची", "भाषा", "करतात", "तर", "संघाचे", "विरोधक", "संघ", "मुक्त", "भारताची", "."]],

   'ner_tags':[["O", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O"],
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], 
             ["B-ORG", "I-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
             ["O", "O", "O", "O", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "O", "B-LOC", "O", "O", "O", "O", "O"], #4
             ["O", "B-PER", "I-PER", "B-PER", "I-PER", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "I-PER", "O", "O", "B-LOC", "O", "O", "B-PER", "O", "O", "O", "O"],
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #6
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "I-ORG", "O", "O", "O"],
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
             ["B-ORG", "I-ORG", "I-ORG", "O", "O", "O", "O", "O", "B-LOC", "O", "B-LOC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #9
             ["B-LOC", "O", "B-LOC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #10
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #11
             ["B-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #12
             ["B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #13
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #14
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
             ["B-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #17
             ["B-LOC", "B-LOC", "O", "B-LOC", "O", "O", "O", "O", "O", "O", "O", "B-LOC", "I-LOC", "O", "O", "O", "O", "B-ORG", "I-ORG", "I-ORG", "O", "O", "O", "O", "O"],
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O"], #19
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
             ["O", "O", "O", "B-LOC", "O", "O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #22
             ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
             ["B-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], #24
             ["B-ORG", "O", "O", "B-ORG", "O", "B-LOC", "O", "O", "O", "B-ORG", "O", "B-ORG", "O", "B-LOC", "O"]]
    }

In [27]:
#NER_labels = ['B-ORG', 'B-MISC', 'I-ORG', 'I-LOC', 'B-LOC', 'I-MISC', 'I-PER', 'O', 'B-PER']
NER_labels = ['B-ORG', 'I-ORG', 'I-LOC', 'B-LOC', 'I-PER', 'O', 'B-PER']
label = ClassLabel(names=NER_labels,num_classes=7)
sequence_feature = Sequence(feature=label, length=-1)
print(sequence_feature)

Sequence(feature=ClassLabel(num_classes=7, names=['B-ORG', 'I-ORG', 'I-LOC', 'B-LOC', 'I-PER', 'O', 'B-PER'], id=None), length=-1, id=None)


In [28]:
features = Features({"tokens": Sequence(feature=Value(dtype='string', id=None)), "ner_tags": sequence_feature})
dataset = Dataset.from_dict(Q1_data, features=features)

In [29]:
Q1_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Q1 dataset",
)

Running tokenizer on Q1 dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Q1 dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Q1 dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Q1 dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [30]:
a = trainer.predict(Q1_dataset)
a.metrics

Class	Precision	Recall	F1-Score
B-LOC	0.15		0.01	0.01
B-ORG	0.00		0.00	0.00
B-PER	0.00		0.00	0.00
I-LOC	0.00		0.00	0.00
I-ORG	0.00		0.00	0.00
I-PER	0.00		0.00	0.00
O	0.03		0.74	0.05
macro f1:	0.00		0.00	0.03


{'test_loss': 5.947450637817383,
 'test_LOC': {'precision': 0.14705882352941177,
  'recall': 0.006165228113440197,
  'f1': 0.011834319526627219,
  'number': 811},
 'test_ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 49},
 'test_PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16},
 'test_overall_precision': 0.08928571428571429,
 'test_overall_recall': 0.005707762557077625,
 'test_overall_f1': 0.01072961373390558,
 'test_overall_accuracy': 0.029473684210526315,
 'test_runtime': 0.5809,
 'test_samples_per_second': 43.039,
 'test_steps_per_second': 3.443}